# Hyperparameter tuning using Keras tuner

Authors: Jerol Soibam, Dimitra-Eirini Diamantidou

## Import libraries

In [1]:
# General libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns

# Tensoflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# Kera tuner libraries
from keras import backend as K
from keras_tuner import RandomSearch
from keras_tuner import Objective
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

# Data preprocessing
import joblib
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

Using TensorFlow backend


In [2]:
# Set random seed state
np.random.seed(42)

## Load data

In [3]:
path_train = "Data//"

file_name = "data.csv"
data = pd.read_csv(path_train + file_name)

## Preprocess data

In [4]:
# Calculate IQR for each column
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the data to exclude outliers
data_no_outliers = data[~((data < lower_bound) | (data > upper_bound)).any(axis=1)]

# Split the dataset between features (X) and targets (y)
X = data_no_outliers.drop(['Unnamed: 0'], axis = 1)[['cell_specific_energy', 'pack_factor', 'eta_batt',
                                                     'SP_batt', 'SP_motor', 'eta_motor', 'SP_gen',
                                                     'eta_gen', 'eta_splitter']]
y = data_no_outliers.drop(['Unnamed: 0'], axis = 1)[['MTOW', "Block_fuel", "El_energy", "Block_energy"]]

x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(X.values)

y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y.values)

## Visualize sampling space

In [ ]:
# # Input space
# scatter_matrix = sns.pairplot(X, diag_kind='hist')
# scatter_matrix.fig.suptitle('Scatter Plot Matrix', y=1.02)

# plot_filename = f'Plots//Xscatter_plot.png'
# scatter_matrix.savefig(plot_filename)

In [ ]:
# # Output space
# scatter_matrix = sns.pairplot(y, diag_kind='hist')
# scatter_matrix.fig.suptitle('Scatter Plot Matrix', y=1.02)

# plot_filename = f'Plots//yscatter_plot.png'
# scatter_matrix.savefig(plot_filename)

## Data split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(x_scaled,
                                                    y_scaled, 
                                                    test_size = 0.20, 
                                                    random_state = 42)

In [6]:
# Save the data scalers in order to use them when employing the surrogate model 
joblib.dump(x_scaler, 'x_scaler.pkl')
joblib.dump(y_scaler, 'y_scaler.pkl')

['y_scaler.pkl']

## Define the NN and hyperparameters

In [7]:
def build_model(hyperparams):
    model = Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))

    num_layers = hyperparams.Int("num_layers", 2, 6)

    for i in range(num_layers):
        model.add(layers.Dense(
            units=hyperparams.Int(f"units_{i+1}", 64, 512, step=32),
            activation=hyperparams.Choice(f"act_{i+1}", ["relu", "tanh"]),
            kernel_regularizer=regularizers.l2(hyperparams.Float(f"l2_{i+1}", 1e-5, 1e-1, sampling="log"))
        ))

    model.add(layers.Dense(y_train.shape[1],))

    batch_size = hyperparams.Int("batch_size", 16, 128, step=16)
    hp_learning_rate = hyperparams.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])

    optimizers_dict = {
        "Adam": keras.optimizers.legacy.Adam(learning_rate=hp_learning_rate),
        "RMSprop": keras.optimizers.legacy.RMSprop(learning_rate=hp_learning_rate)
    }

    hp_optimizers = hyperparams.Choice('optimizer', values=["Adam", "RMSprop"])

    model.compile(
        loss='mean_squared_error',
        optimizer=optimizers_dict[hp_optimizers],
        metrics=["mean_squared_error"]
    )

    return model

## Callbacks

In [8]:
# Callback for writing checkpoints and saving the weights
path_checkpoint = 'weights//hyperparameter_search'
callback_checpoint = ModelCheckpoint(filepath=path_checkpoint, 
                                    monitor ='val_loss',
                                    verbose=1,
                                    save_weights_only = True,
                                    save_best_only = True)

# Callback for early stopping the optimization when performance worsens on the validation-set
callback_early_stopping = EarlyStopping(monitor = 'val_loss',
                                       patience =60,
                                       verbose=1)

# Callback for TensorBoard log during training
tensorboard_log_dir = "./random_search/tb1_logs"
os.makedirs(tensorboard_log_dir, exist_ok=True)

callback_tensorboard = TensorBoard(tensorboard_log_dir)

# Callback for learning rate
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                      factor =0.1,
                                      min_lr = 1e-5,
                                      patience=30,
                                      verbose=1)

In [9]:
callbacks = [callback_early_stopping,
            callback_checpoint,
            callback_tensorboard,
            callback_reduce_lr]

## Run the hyperparameter search

In [10]:
tuner1 = RandomSearch(
    hypermodel=build_model,
    objective=Objective(name="val_mean_squared_error", direction="min"),
    max_trials=5,
    project_name="Regression",
    overwrite=True,
)

tuner1.search(X_train, y_train, epochs=2000, validation_split = 0.1, callbacks=callbacks)

Trial 5 Complete [00h 12m 30s]
val_mean_squared_error: 3.0502495064865798e-05

Best val_mean_squared_error So Far: 1.030353541864315e-05
Total elapsed time: 01h 08m 37s


## Extract the best performing model

In [16]:
best_params = tuner1.get_best_hyperparameters()
best_params[0].values

{'num_layers': 6,
 'units_1': 416,
 'act_1': 'relu',
 'l2_1': 0.057138316142351334,
 'units_2': 448,
 'act_2': 'tanh',
 'l2_2': 1.5631858997681786e-05,
 'batch_size': 64,
 'learning_rate': 0.0001,
 'optimizer': 'Adam',
 'units_3': 64,
 'act_3': 'relu',
 'l2_3': 1e-05,
 'units_4': 64,
 'act_4': 'relu',
 'l2_4': 1e-05,
 'units_5': 64,
 'act_5': 'relu',
 'l2_5': 1e-05,
 'units_6': 64,
 'act_6': 'relu',
 'l2_6': 1e-05}

In [17]:
best_model = tuner1.get_best_models()[0]
best_model.summary()

best_model.save('surrogate_model.keras', save_format='tf', signatures=None)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 416)               4160      
                                                                 
 dense_1 (Dense)             (None, 448)               186816    
                                                                 
 dense_2 (Dense)             (None, 64)                28736     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 4)                 2

## Postprocessing

In [12]:
y_pred = best_model.predict(X_test)

y_pred_inverse = y_scaler.inverse_transform(y_pred)
y_test_inverse = y_scaler.inverse_transform(y_test)

ynames = ['MTOW', "Block_fuel", "El_energy", "Block_energy"]
ylen = len(ynames)

68/68 [==============================] - 0s 757us/step


In [13]:
# Regression plots
for element in range(ylen):

    plt.figure(figsize = [5,5])
    plt.scatter(y_test_inverse[:,element], y_pred_inverse[:,element], alpha=0.3)
    plt.plot([np.min(y_test_inverse[:,element]),np.max(y_test_inverse[:,element])],[np.min(y_test_inverse[:,element]),np.max(y_test_inverse[:,element])], "k-")
    plt.xlabel(f"{ynames[element]} True")
    plt.ylabel(f"{ynames[element]} Pred")
    plt.tight_layout()
    plt.savefig(f"plots//{ynames[element]}.png")
    plt.close()

In [14]:
# R2 score
for element in range(ylen):
    r2 = (r2_score(y_test_inverse[:,element],y_pred_inverse[:,element]))
    print(f"R2 for {ynames[element]} is {r2}")

R2 for MTOW is 0.9996771356351211
R2 for Block_fuel is 0.9998877482223514
R2 for El_energy is 0.9997183175245212
R2 for Block_energy is 0.9997704006168446


In [15]:
# RMSE
for element in range(ylen):
    rmse = np.sqrt(mean_squared_error(y_test_inverse[:,element],y_pred_inverse[:,element]))
    print(f"RMSE for {ynames[element]} is {rmse}")

RMSE for MTOW is 0.2962613881881074
RMSE for Block_fuel is 0.15648177147553274
RMSE for El_energy is 0.9353793440262083
RMSE for Block_energy is 1.8290747766389812
